In [1]:
pip install pandas scikit-learn numpy Pillow keras_tuner


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image

# Define the root directory where your image folders are located
root_directory = "PartB_DFU_dataset - Copy"

# Initialize lists to store image paths and corresponding class labels for both datasets
image_paths_ischaemia = []
categories_ischaemia = []
image_paths_infection = []
categories_infection = []

# Iterate over each class and its subdirectories
for class_name in ["Infection", "Ischaemia"]:
    for augmentation_type in ["Aug-Negative", "Aug-Positive"]:
        folder_path = os.path.join(root_directory, class_name, augmentation_type)
        category = f"{class_name.lower()}{'pov' if 'Positive' in augmentation_type else 'neg'}"
        
        # Iterate over image files in the current directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".jpg"):  # Assuming images are jpg format
                image_path = os.path.join(folder_path, file_name)
                if class_name == "Ischaemia":
                    image_paths_ischaemia.append(image_path)
                    categories_ischaemia.append("ischemia" if "Positive" in augmentation_type else "non-ischemia")
                elif class_name == "Infection":
                    image_paths_infection.append(image_path)
                    categories_infection.append("infection" if "Positive" in augmentation_type else "non-infection")

# Create DataFrames for each dataset
df_ischaemia = pd.DataFrame({"category": categories_ischaemia, "image_path": image_paths_ischaemia})
df_infection = pd.DataFrame({"category": categories_infection, "image_path": image_paths_infection})

# Label encoding for Ischaemia dataset
label_encoder_ischaemia = LabelEncoder()
df_ischaemia['Class_Label'] = label_encoder_ischaemia.fit_transform(df_ischaemia['category'])
print("Ischaemia Class Mapping:")
for class_label, numerical_label in zip(df_ischaemia['category'].unique(), df_ischaemia['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Label encoding for Infection dataset
label_encoder_infection = LabelEncoder()
df_infection['Class_Label'] = label_encoder_infection.fit_transform(df_infection['category'])
print("Infection Class Mapping:")
for class_label, numerical_label in zip(df_infection['category'].unique(), df_infection['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Shuffle both DataFrames
df_ischaemia = df_ischaemia.sample(frac=1).reset_index(drop=True)
df_infection = df_infection.sample(frac=1).reset_index(drop=True)

# Helper function to load and process images
def load_images(df):
    images = []
    target_labels = []   
    for index, row in df.iterrows():
        image = Image.open(row['image_path'])
        image_array = np.array(image.resize((256, 256)))  # Resize image to fit model input size
        images.append(image_array)
        target_labels.append(row['Class_Label'])
    return np.array(images), np.array(target_labels)

# Load images for both datasets
images_ischaemia, target_labels_ischaemia = load_images(df_ischaemia)
images_infection, target_labels_infection = load_images(df_infection)

print("Shape of Ischaemia images array:", images_ischaemia.shape)
print("Shape of Ischaemia target labels array:", target_labels_ischaemia.shape)
print("Shape of Infection images array:", images_infection.shape)
print("Shape of Infection target labels array:", target_labels_infection.shape)

# Split the Ischaemia dataset
X_train_ischaemia, X_test_ischaemia, y_train_ischaemia, y_test_ischaemia = train_test_split(
    images_ischaemia, target_labels_ischaemia, test_size=0.3, random_state=42)
X_val_ischaemia, X_test_ischaemia, y_val_ischaemia, y_test_ischaemia = train_test_split(
    X_test_ischaemia, y_test_ischaemia, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

# Split the Infection dataset
X_train_infection, X_test_infection, y_train_infection, y_test_infection = train_test_split(
    images_infection, target_labels_infection, test_size=0.3, random_state=42)
X_val_infection, X_test_infection, y_val_infection, y_test_infection = train_test_split(
    X_test_infection, y_test_infection, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

print("Ischaemia Training set shape:", X_train_ischaemia.shape, y_train_ischaemia.shape)
print("Ischaemia Validation set shape:", X_val_ischaemia.shape, y_val_ischaemia.shape)
print("Ischaemia Test set shape:", X_test_ischaemia.shape, y_test_ischaemia.shape)
print("Infection Training set shape:", X_train_infection.shape, y_train_infection.shape)
print("Infection Validation set shape:", X_val_infection.shape, y_val_infection.shape)
print("Infection Test set shape:", X_test_infection.shape, y_test_infection.shape)

Ischaemia Class Mapping:
non-ischemia: 1
ischemia: 0
Infection Class Mapping:
non-infection: 1
infection: 0
Shape of Ischaemia images array: (9870, 256, 256, 3)
Shape of Ischaemia target labels array: (9870,)
Shape of Infection images array: (5890, 256, 256, 3)
Shape of Infection target labels array: (5890,)
Ischaemia Training set shape: (6909, 256, 256, 3) (6909,)
Ischaemia Validation set shape: (2220, 256, 256, 3) (2220,)
Ischaemia Test set shape: (741, 256, 256, 3) (741,)
Infection Training set shape: (4123, 256, 256, 3) (4123,)
Infection Validation set shape: (1325, 256, 256, 3) (1325,)
Infection Test set shape: (442, 256, 256, 3) (442,)


In [3]:
import numpy as np
from keras_tuner import HyperModel, HyperParameters
import keras_tuner as kt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, TimeDistributed, LSTM
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, SGD, RMSprop


def build_model(hp):
    model = Sequential()
    
    # First Convolutional Block
    model.add(Conv2D(filters=int(hp.Int('conv_1_filters', min_value=32, max_value=64, step=16)),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu',
                     input_shape=(256, 256, 3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    # Second Convolutional Block
    model.add(Conv2D(filters=int(hp.Int('conv_2_filters', min_value=64, max_value=128, step=16)),
                     kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    # Third Convolutional Block
    model.add(Conv2D(filters=int(hp.Int('conv_3_filters', min_value=128, max_value=256, step=16)),
                     kernel_size=hp.Choice('conv_3_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    model.add(TimeDistributed(Flatten()))
    
    # LSTM Layers
    model.add(LSTM(int(hp.Int('lstm_units_1', min_value=80, max_value=120, step=10)), 
                   dropout=hp.Float('dropout_1', min_value=0.2, max_value=0.3, step=0.1), 
                   return_sequences=True))
    model.add(LSTM(int(hp.Int('lstm_units_2', min_value=50, max_value=70, step=10)), 
                   dropout=hp.Float('dropout_2', min_value=0.2, max_value=0.3, step=0.1), 
                   return_sequences=True))
    model.add(LSTM(int(hp.Int('lstm_units_3', min_value=20, max_value=40, step=10)), 
                   dropout=0.2, 
                   return_sequences=False))
    
    # Fully Connected Layers
    model.add(Dense(int(hp.Int('dense_1_units', min_value=128, max_value=512, step=64)), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.3, max_value=0.5, step=0.1)))
    model.add(BatchNormalization())
    
    model.add(Dense(int(hp.Int('dense_2_units', min_value=64, max_value=256, step=64)), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.2, max_value=0.4, step=0.1)))
    model.add(BatchNormalization())
    
    model.add(Dense(3, activation='softmax'))
    
    # Optimizer
    learning_rate_params = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer_params = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'Adadelta', 'Adagrad'])
    
    if optimizer_params == 'adam':
        optimizer = Adam(learning_rate=learning_rate_params)
    elif optimizer_params == 'sgd':
        optimizer = SGD(learning_rate=learning_rate_params)
    elif optimizer_params == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate_params)
    elif optimizer_params == 'Adagrad':
        optimizer = Adagrad(learning_rate=learning_rate_params)
    elif optimizer_params == 'Adadelta':
        optimizer = Adadelta(learning_rate=learning_rate_params)
    else:
        raise ValueError(f"Invalid optimizer choice: {optimizer_choice}")
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


2024-10-02 10:27:34.721998: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 10:27:34.737626: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 10:27:34.742342: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 10:27:34.755376: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:

import numpy as np
import random

# EHO Parameters
num_clans = 1  # Since population size is 5, let's use 1 clan with 5 elephants for simplicity
num_elephants_per_clan = 5
max_generations = 5
epochs = 10  # Number of training epochs

# Boundaries for the hyperparameters
bounds = {
    'conv_1_filters': [32, 64],
    'conv_2_filters': [64, 128],
    'conv_3_filters': [128, 256],
    'lstm_units_1': [80, 120],
    'lstm_units_2': [50, 70],
    'lstm_units_3': [20, 40],
    'dropout_1': [0.2, 0.3],
    'dropout_2': [0.2, 0.3],
    'dropout_3': [0.3, 0.5],
    'dropout_4': [0.2, 0.4],
    'dense_1_units': [128, 512],
    'dense_2_units': [64, 256],
    'learning_rate': [1e-4, 1e-2]
}

# Clan structure
def initialize_population():
    population = []
    for _ in range(num_clans):
        clan = []
        for _ in range(num_elephants_per_clan):
            elephant = {param: random.uniform(bounds[param][0], bounds[param][1]) for param in bounds}
            clan.append(elephant)
        population.append(clan)
    return population

# Fitness evaluation (e.g., model accuracy)
def evaluate_fitness(elephant):
    # Define hyperparameters using the current elephant's values
    hp = HyperParameters()
    for param, value in elephant.items():
        hp.Fixed(param, value)
    
    # Build and compile the model
    model = build_model(hp)
    
    # Train the model
    history = model.fit(
        X_train_ischaemia, y_train_ischaemia, 
        validation_data=(X_val_ischaemia, y_val_ischaemia), 
        epochs=epochs,  # Set epochs to 10
        verbose=0  # Suppress detailed logs
    )
    
    # Retrieve the final epoch's metrics
    training_accuracy = history.history['accuracy'][-1]
    training_loss = history.history['loss'][-1]
    validation_accuracy = history.history['val_accuracy'][-1]
    validation_loss = history.history['val_loss'][-1]
    
    # Print the training and validation metrics
    print(f"Training Accuracy: {training_accuracy}, Training Loss: {training_loss}")
    print(f"Validation Accuracy: {validation_accuracy}, Validation Loss: {validation_loss}")
    
    # Return validation accuracy as fitness score
    return validation_accuracy

# Clan updating operator
def clan_update(clan, matriarch):
    for elephant in clan:
        for param in bounds:
            beta = np.random.uniform(0, 1)
            elephant[param] = beta * matriarch[param] + (1 - beta) * elephant[param]
    return clan

# Separating operator
def separating_operator(clan):
    worst_elephant = sorted(clan, key=evaluate_fitness)[0]
    for param in worst_elephant:
        worst_elephant[param] = random.uniform(bounds[param][0], bounds[param][1])
    return clan

# EHO main loop
def eho_optimization():
    population = initialize_population()
    
    best_solution = None
    best_fitness = -float('inf')
    
    for generation in range(max_generations):
        for clan in population:
            # Find the matriarch (best elephant)
            matriarch = max(clan, key=evaluate_fitness)
            # Update clan members
            clan = clan_update(clan, matriarch)
            # Perform separating operation on worst elephant
            clan = separating_operator(clan)
        
        # Find the best solution in the current generation
        current_best = max([max(clan, key=evaluate_fitness) for clan in population], key=evaluate_fitness)
        current_best_fitness = evaluate_fitness(current_best)
        
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_solution = current_best
        
        print(f"Generation {generation+1}, Best Fitness: {current_best_fitness}")
    
    return best_solution

# Run the EHO optimization
best_hyperparameters = eho_optimization()

# Print the best hyperparameters
print("Best Hyperparameters after EHO Optimization:")
print(best_hyperparameters)

# Evaluate the best model with test accuracy
def evaluate_best_hyperparameters(best_hyperparameters):
    # Define the best hyperparameters using the final best solution
    hp = HyperParameters()
    for param, value in best_hyperparameters.items():
        hp.Fixed(param, value)
    
    # Build and compile the model with best hyperparameters
    model = build_model(hp)
    
    # Train the model with best hyperparameters
    history = model.fit(
        X_train_ischaemia, y_train_ischaemia, 
        validation_data=(X_val_ischaemia, y_val_ischaemia), 
        epochs=epochs,  # Keep it consistent with 10 epochs
        verbose=1  # Show detailed logs now
    )
    
    # Print the final metrics for the best hyperparameters
    training_accuracy = history.history['accuracy'][-1]
    training_loss = history.history['loss'][-1]
    validation_accuracy = history.history['val_accuracy'][-1]
    validation_loss = history.history['val_loss'][-1]
    
    print(f"Best Model Training Accuracy: {training_accuracy}, Training Loss: {training_loss}")
    print(f"Best Model Validation Accuracy: {validation_accuracy}, Validation Loss: {validation_loss}")
    
    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test_ischaemia, y_test_ischaemia, verbose=1)
    print(f"Best Model Test Accuracy: {test_accuracy}, Test Loss: {test_loss}")

# Evaluate the best model
evaluate_best_hyperparameters(best_hyperparameters)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-02 10:27:40.126834: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22066 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2024-10-02 10:27:40.131405: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22066 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:21:00.0, compute capability: 8.9
2024-10-02 10:27:47.171055: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
W0000 00:00:

Training Accuracy: 0.8611955642700195, Training Loss: 0.32225072383880615
Validation Accuracy: 0.8189189434051514, Validation Loss: 0.38815516233444214


W0000 00:00:1727864983.110673   63377 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727864983.111929   63377 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727864983.112815   63377 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727864983.113696   63377 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727864983.114592   63377 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727864983.115503   63377 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727864983.117104   63377 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727864983.118705   63377 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727864983.120325   63377 gp

Training Accuracy: 0.8419452905654907, Training Loss: 0.36308375000953674
Validation Accuracy: 0.7720720767974854, Validation Loss: 0.5523510575294495


W0000 00:00:1727865100.996507   63381 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727865100.997722   63381 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727865100.998394   63381 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727865100.999038   63381 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727865100.999771   63381 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727865101.000512   63381 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727865101.001672   63381 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727865101.002850   63381 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727865101.003926   63381 gp

...

W0000 00:00:1727871107.399306   63372 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871107.400841   63372 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871107.402910   63372 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871107.405269   63372 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871107.407079   63372 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871107.408934   63372 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871107.410050   63372 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871107.411722   63372 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871107.413734   63372 gp

Training Accuracy: 0.8703140616416931, Training Loss: 0.31125882267951965
Validation Accuracy: 0.8504504561424255, Validation Loss: 0.36889663338661194


W0000 00:00:1727871212.342972   63376 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871212.345487   63376 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871212.347290   63376 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871212.349493   63376 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871212.352511   63376 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871212.353806   63376 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871212.355584   63376 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871212.357386   63376 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871212.358900   63376 gp

Training Accuracy: 0.8959328532218933, Training Loss: 0.24160632491111755
Validation Accuracy: 0.8851351141929626, Validation Loss: 0.2616942524909973
Training Accuracy: 0.8978144526481628, Training Loss: 0.24659329652786255
Validation Accuracy: 0.912162184715271, Validation Loss: 0.22469601035118103
Training Accuracy: 0.8856564164161682, Training Loss: 0.2731779217720032
Validation Accuracy: 0.8716216087341309, Validation Loss: 0.3005050718784332
Training Accuracy: 0.859748125076294, Training Loss: 0.3363683223724365
Validation Accuracy: 0.8540540337562561, Validation Loss: 0.4033489525318146


W0000 00:00:1727871657.410839   63371 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871657.412295   63371 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871657.413059   63371 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871657.413815   63371 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871657.414616   63371 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871657.415409   63371 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871657.416679   63371 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871657.417916   63371 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727871657.419183   63371 gp